In [ ]:
from sklearn import tree
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
from IPython.display import display
from IPython.display import SVG
from graphviz import Source
from IPython.display import HTML
style = "<style>svg{width:30% !important;height:30% !important;}</style>"
# HTML(style)

In [ ]:
# https://www.kaggle.com/c/titanic/data ==> explore
titanic_data = pd.read_csv("/home/octopus/Documents/2scripts/ScientificPython/3_ML/Lecture10_DT/files/train.csv")
print(titanic_data.shape)
titanic_data.head()

TASK: train a decision tree that will predict whether the passenger survived or did not survive, based on the passenger data from the train.csv dataset.


In [ ]:
titanic_data.isnull().head()

In [ ]:
titanic_data.isnull().sum()

In [ ]:
titanic_data.Age.median()

In [ ]:
titanic_data = titanic_data.fillna({'Age': titanic_data.Age.median()})

In [ ]:
titanic_data.isnull().sum()

In [ ]:
# Which columns will we delete?

In [ ]:
titanic_data.head()

In [ ]:
# What does axis = 1 mean?
X = titanic_data.drop(["PassengerId", "Survived", "Name", "Ticket", "Cabin"], axis = 1)

In [ ]:
X

In [ ]:
X.isnull().sum()

In [ ]:
y = titanic_data.Survived
y.head()

In [ ]:
clf = tree.DecisionTreeClassifier(criterion='entropy')

In [ ]:
clf.fit(X,y)

In [ ]:
X

In [ ]:
X = pd.get_dummies(X)

In [ ]:
X.head()

In [ ]:
clf.fit(X, y)

In [ ]:
# graph = Source(tree.export_graphviz(clf, out_file=None,
#                                    feature_names=list(X),
#                                    class_names=['Negative','Positive'],
#                                    filled = True))
# display(SVG(graph.pipe(format='svg')))

In [ ]:
import os
dotfile = open("dtree2.dot", 'w')
tree.export_graphviz(clf, 
                     out_file = dotfile, 
                     feature_names = list(X),
                     class_names=['Negative','Positive'],
                     filled = True)
dotfile.close()
os.system("dot -Tpng dtree2.dot -o dtree2.png")

In [ ]:
clf.score(X, y)

In [ ]:
# we studied to classify this particular dataset but we didn't study to find rules/patterns

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test =  train_test_split(X, 
                                                     y, 
                                                     test_size=0.30, 
                                                     random_state=42) # seed

# 30% of the data will go into the test sample, the rest -- in training

In [ ]:
X_train.head()

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
X.shape

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
clf.score(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
# looks like overfitting
# we want to avoid it. we can do it by several ways:

In [ ]:
from sklearn import set_config
set_config(print_changed_only=False)
clf.fit(X_train, y_train)

In [ ]:
clf = tree.DecisionTreeClassifier(criterion='entropy', 
                                  max_depth=3)
# we are trying to avoid overfitting

# The maximum depth of the tree. 

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
clf.score(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
# Much better but whether this max_depth is optimal?

In [ ]:
max_depth_values = range(1, 100)

In [ ]:
scores_data = pd.DataFrame()

In [ ]:
for max_depth in max_depth_values:
    
    clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=max_depth)
    
    clf.fit(X_train, y_train)
    
    train_score = clf.score(X_train, y_train)
    
    test_score = clf.score(X_test, y_test)
    
    temp_score_data = pd.DataFrame({'max_depth': [max_depth],
                                    'train_score': [train_score],
                                    'test_score': [test_score]})
    
    scores_data = scores_data.append(temp_score_data)

In [ ]:
scores_data.head()

In [ ]:
scores_data_long = pd.melt(scores_data,
                           id_vars=['max_depth'],
                           value_vars=['train_score', 'test_score'],
                           var_name='set_type',
                           value_name='score')

In [ ]:
scores_data_long.head()

In [ ]:
sns.lineplot(x="max_depth", 
             y="score", 
             hue="set_type",
             data=scores_data_long)

In [ ]:
# There is one more problem: we always have the same test data.

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
clf = tree.DecisionTreeClassifier(criterion='entropy', 
                                  max_depth=4) 

In [ ]:
cross_val_score(clf, 
                X, 
                y, 
                cv=5,
                verbose=True) # validation score for test -> by default split 5 times, can ve changed `cv=5`

In [ ]:
# Train: [2 3 4 5 6 7 8 9] | test: [0 1]
# Train: [0 1 4 5 6 7 8 9] | test: [2 3]
# Train: [0 1 2 3 6 7 8 9] | test: [4 5]
# Train: [0 1 2 3 4 5 8 9] | test: [6 7]
# Train: [0 1 2 3 4 5 6 7] | test: [8 9]

In [ ]:
cross_val_score(clf, X, y).mean()

In [ ]:
from sklearn.model_selection import cross_validate

In [ ]:
cross_validate(clf, X, y) # the full output, not just score only

In [ ]:
scores_data = pd.DataFrame()

In [ ]:
for max_depth in max_depth_values:
    
    clf = tree.DecisionTreeClassifier(criterion='entropy', 
                                      max_depth=max_depth)
    
    # 1
    clf.fit(X_train, y_train)
    # train
    train_score = clf.score(X_train, y_train)
    # test
    test_score = clf.score(X_test, y_test)
    
    # 2
    mean_cross_val_score = cross_val_score(clf, X, y).mean()
    
    temp_score_data = pd.DataFrame({'max_depth': [max_depth],
                                    'train_score': [train_score],
                                    'test_score': [test_score],
                                    'cross_val_score': [mean_cross_val_score]})
    
    scores_data = scores_data.append(temp_score_data)

In [ ]:
scores_data.head()

In [ ]:
scores_data_long = pd.melt(scores_data, 
                           id_vars=['max_depth'], 
                           value_vars=['train_score', 'test_score', 'cross_val_score'],
                           var_name='set_type', 
                           value_name='score')

In [ ]:
scores_data_long.head()

In [ ]:
sns.lineplot(x="max_depth", 
             y="score", 
             hue="set_type", 
             data=scores_data_long)

In [ ]:
scores_data_long.query("set_type == 'cross_val_score'").head(20)

In [ ]:
qq = scores_data_long.query("set_type == 'cross_val_score'")

In [ ]:
qq.score.max() # optimal depth is 7

In [ ]:
best_clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=7)

In [ ]:
best_clf.fit(X_train, y_train)



In [ ]:
best_clf.score(X_test, y_test)